------**CAPA 1: ORIENTACIÓN AL TIPO DE SERVICIO**--------

In [ ]:
# =========================
# 0) Librerías (Colab)
# =========================

# Si estás en Colab, descomenta esta sección para instalar dependencias.
# Recomendado: ejecutar una sola vez al inicio del notebook.

!pip -q install pandas numpy scikit-learn matplotlib seaborn tqdm unidecode regex
!pip -q install sentence-transformers umap-learn hdbscan bertopic
!pip -q install nltk spacy
!python -m spacy download es_core_news_sm -q

import os
import re
import regex as re2
import unicodedata
from datetime import datetime

import numpy as np
import pandas as pd

from tqdm.auto import tqdm

# Visualización (opcional por ahora)
import matplotlib.pyplot as plt

# Preprocesamiento y utilidades ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix

# NLP
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords

import spacy
nlp_es = spacy.load("es_core_news_sm", disable=["ner", "parser"])  # liviano para limpieza

from unidecode import unidecode

# Embeddings y tópicos
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import umap
import hdbscan

print("Librerías cargadas OK")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 100.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


Librerías cargadas OK


In [ ]:
# =========================
# 1) Carga de datos
# =========================

ruta_archivo = "/content/data.csv"

df = pd.read_csv(ruta_archivo)

print("Dimensiones del dataset:", df.shape)
df.head()


Dimensiones del dataset: (30000, 2)


,CodHw,Descripcion
0,1558176,Se aplaza
1,1558176,Se cambia fecha de compromiso.
2,1627445,Situación Reportada:\r\n\r\n-se da plazo a ia ...
3,1627445,Situación Reportada:\r\n\r\n-se da plazo a ia ...
4,1630429,Situación Reportada:\r\n\r\nIndicaciones:


In [ ]:
# =========================
# 1.1) Revisión de columnas
# =========================

df.columns


Index(['CodHw', 'Descripcion'], dtype='object')

In [ ]:
# =========================
# 1) Subconjunto de trabajo
# =========================

df_work = df[["CodHw", "Descripcion"]].copy()

print("Registros totales:", df_work.shape[0])
df_work.head()



Registros totales: 30000


,CodHw,Descripcion
0,1558176,Se aplaza
1,1558176,Se cambia fecha de compromiso.
2,1627445,Situación Reportada:\r\n\r\n-se da plazo a ia ...
3,1627445,Situación Reportada:\r\n\r\n-se da plazo a ia ...
4,1630429,Situación Reportada:\r\n\r\nIndicaciones:


In [ ]:
# =========================
# 1.1) Calidad básica del texto
# =========================

# Conteo de nulos
print("Nulos en Descripcion:", df_work["Descripcion"].isna().sum())

# Longitud del texto
df_work["len_texto"] = df_work["Descripcion"].astype(str).str.len()

df_work["len_texto"].describe()


Nulos en Descripcion: 90


,len_texto
count,30000.000000
mean,806.159667
std,1795.651445
min,1.000000
25%,66.000000
50%,147.000000
75%,580.000000
max,32766.000000


In [ ]:
# =========================
# 1.2) Muestra aleatoria
# =========================

df_work.sample(5, random_state=42)[["CodHw", "Descripcion"]]


,CodHw,Descripcion
2308,2066971,Situación Reportada:\r\n\r\n-se apoya en repro...
22404,2074964,Cliente no puede ingresar a su conexion remota...
23397,2075347,Solicitud de asistencia en Línea desde el Chat...
25058,2075978,'-SE LLAMA IVR.\r\n\r\n-CX REPORTA ERROR EN PA...
2664,2067111,Situación Reportada:cx necesita actualizar sis...


In [ ]:
# =========================
# 2) Limpieza y normalización controlada
# =========================

# 2.1 Eliminar nulos
df_work = df_work.dropna(subset=["Descripcion"]).reset_index(drop=True)

print("Registros luego de eliminar nulos:", df_work.shape[0])


Registros luego de eliminar nulos: 29910


In [ ]:
# =========================
# 2.2 Función de normalización
# =========================

stopwords_es = set(stopwords.words("spanish"))

def normalizar_texto(texto):
    texto = str(texto).lower()

    # eliminar saltos de línea múltiples
    texto = re.sub(r"\r\n|\n|\r", " ", texto)

    # eliminar espacios repetidos
    texto = re.sub(r"\s+", " ", texto).strip()

    # normalizar acentos
    texto = unidecode(texto)

    return texto

df_work["texto_norm"] = df_work["Descripcion"].apply(normalizar_texto)

df_work[["Descripcion", "texto_norm"]].head(3)


,Descripcion,texto_norm
0,Se aplaza,se aplaza
1,Se cambia fecha de compromiso.,se cambia fecha de compromiso.
2,Situación Reportada:\r\n\r\n-se da plazo a ia ...,situacion reportada: -se da plazo a ia mdo


In [ ]:
# =========================
# 2.3 Control de longitud extrema
# =========================

MAX_CHARS = 3000  # suficiente para capturar contexto real

df_work["texto_norm"] = df_work["texto_norm"].apply(
    lambda x: x[:MAX_CHARS] if len(x) > MAX_CHARS else x
)

df_work["len_texto_norm"] = df_work["texto_norm"].str.len()
df_work["len_texto_norm"].describe()


,len_texto_norm
count,29910.000000
mean,572.447309
std,894.011415
min,1.000000
25%,65.000000
50%,144.000000
75%,561.000000
max,3000.000000


In [ ]:
# =========================
# 2.4 Filtro de textos demasiado cortos
# =========================

MIN_CHARS = 30

df_work = df_work[df_work["len_texto_norm"] >= MIN_CHARS].reset_index(drop=True)

print("Registros luego de filtro por longitud mínima:", df_work.shape[0])


Registros luego de filtro por longitud mínima: 27543


In [ ]:
# =========================
# 3) Marcaje de señales de negocio
# =========================

# 3.1 Diccionarios de señales
patrones_ia = r"\bia\b|requiere desarrollo|pasa a desarrollo|derivado a ia"
patrones_qa = r"\bqa\b|calidad|validacion qa"
patrones_cx = r"\bcx\b|cliente valida|respuesta cliente"

patrones_gestion = r"se atiende|se aplaza|se cierra|pendiente|reasigna|se asigna|plazo|fecha compromiso"

patrones_responsable = r"\b[a-z]{3}\b"  # codigos tipo LLG, AGV, etc.

# 3.2 Función de detección
def detectar_patrones(texto, patron):
    return int(bool(re.search(patron, texto)))

# 3.3 Crear flags
df_work["flag_ia"] = df_work["texto_norm"].apply(lambda x: detectar_patrones(x, patrones_ia))
df_work["flag_qa"] = df_work["texto_norm"].apply(lambda x: detectar_patrones(x, patrones_qa))
df_work["flag_cx"] = df_work["texto_norm"].apply(lambda x: detectar_patrones(x, patrones_cx))
df_work["flag_gestion"] = df_work["texto_norm"].apply(lambda x: detectar_patrones(x, patrones_gestion))
df_work["flag_responsable"] = df_work["texto_norm"].apply(lambda x: detectar_patrones(x, patrones_responsable))

df_work[["flag_ia", "flag_qa", "flag_cx", "flag_gestion", "flag_responsable"]].head()


,flag_ia,flag_qa,flag_cx,flag_gestion,flag_responsable
0,0,0,0,0,0
1,1,0,0,1,1
2,1,0,0,1,1
3,0,0,0,0,0
4,0,0,0,1,1


In [ ]:
# =========================
# 3.4 Resumen de señales
# =========================

df_work[["flag_ia", "flag_qa", "flag_cx", "flag_gestion"]].mean().sort_values(ascending=False)


,0
flag_cx,0.245979
flag_gestion,0.118760
flag_ia,0.047090
flag_qa,0.007116


In [ ]:
# =========================
# 3.5 Muestra con flags
# =========================

df_work.sample(5, random_state=1)[
    ["CodHw", "texto_norm", "flag_ia", "flag_qa", "flag_cx", "flag_gestion"]
]


,CodHw,texto_norm,flag_ia,flag_qa,flag_cx,flag_gestion
10638,2070664,situacion reportada: -se atiende llamado desde...,0,0,1,1
2741,2067260,situacion reportada: se atachan 3 bd se ejecut...,0,0,0,0
12321,2071368,situacion reportada: -. se llama a cliente y s...,0,0,0,0
11193,2070885,'-cliente responde correo indicando que no est...,0,0,0,0
3091,2067415,solicitud de asistencia en linea desde el chat...,0,0,0,0


In [ ]:
# =========================
# 3.6 Diccionario de módulos Softland
# =========================

patrones_modulos = {
    "CW": r"\bcw\b|contabilidad|contable|comprobante|asiento|plan de cuentas",
    "IW": r"\biw\b|inventario|bodega|stock|movimiento|producto",
    "SW": r"\bsw\b|remuneracion|rrhh|personal|liquidacion|haberes",
    "NW": r"\bnw\b|facturacion|venta|nota de venta|boleta|factura",
    "OW": r"\bow\b|orden de compra|oc\b",
    "HW": r"\bhw\b|helpdesk|soporte"
}


In [ ]:
# =========================
# 3.7 Función de detección de módulos
# =========================

def detectar_modulo(texto, patron):
    return int(bool(re.search(patron, texto)))


In [ ]:
# =========================
# 3.8 Flags por módulo
# =========================

for modulo, patron in patrones_modulos.items():
    df_work[f"mod_{modulo}"] = df_work["texto_norm"].apply(
        lambda x: detectar_modulo(x, patron)
    )

df_work[[c for c in df_work.columns if c.startswith("mod_")]].head()


,mod_CW,mod_IW,mod_SW,mod_NW,mod_OW,mod_HW
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [ ]:
# =========================
# 3.9 Distribución de módulos
# =========================

df_work[[c for c in df_work.columns if c.startswith("mod_")]].mean().sort_values(ascending=False)


,0
mod_HW,0.310896
mod_NW,0.128345
mod_CW,0.089460
mod_IW,0.088952
mod_SW,0.067531
mod_OW,0.020332


In [ ]:
# =========================
# 3.10 Muestra con módulos detectados
# =========================

df_work.sample(5, random_state=7)[
    ["CodHw", "texto_norm"] + [c for c in df_work.columns if c.startswith("mod_")]
]


,CodHw,texto_norm,mod_CW,mod_IW,mod_SW,mod_NW,mod_OW,mod_HW
20262,2074853,situacion reportada: -se atiende llamado desde...,0,0,0,1,0,0
24437,2076579,cliente al subir archivo a previred entrega me...,0,0,0,0,0,0
26446,2077459,76.053.885-k - carolina pino cpino@dercorp.cl ...,1,0,0,0,0,0
217,2039157,situacion reportada: se extinden fechas cx aun...,0,0,0,0,0,0
14214,2072256,situacion reportada: se toma comunicacion con ...,0,0,0,0,0,0


In [ ]:
# =========================
# 4.1 Patrones de ruido operativo
# =========================

patrones_ruido = [
    r"se atiende.*?$",
    r"se aplaza.*?$",
    r"se cierra.*?$",
    r"pendiente.*?$",
    r"reasigna.*?$",
    r"se asigna.*?$",
    r"fecha compromiso.*?$",
    r"plazo.*?$",
    r"derivado.*?$"
]


In [ ]:
# =========================
# 4.2 Limpieza controlada de gestión
# =========================

def limpiar_gestion(texto):
    texto_limpio = texto
    for patron in patrones_ruido:
        texto_limpio = re.sub(patron, " ", texto_limpio)
    texto_limpio = re.sub(r"\s+", " ", texto_limpio).strip()
    return texto_limpio


In [ ]:
# =========================
# 4.3 Texto final para análisis
# =========================

df_work["texto_analitico"] = df_work["texto_norm"].apply(limpiar_gestion)

df_work["len_texto_analitico"] = df_work["texto_analitico"].str.len()

df_work["len_texto_analitico"].describe()


,len_texto_analitico
count,27543.000000
mean,571.771049
std,892.268553
min,0.000000
25%,66.000000
50%,140.000000
75%,572.000000
max,3000.000000


In [ ]:
# =========================
# 4.4 Comparación antes / después
# =========================

df_work.sample(5, random_state=10)[
    ["Descripcion", "texto_norm", "texto_analitico"]
]


,Descripcion,texto_norm,texto_analitico
20911,Solicitud de asistencia en Línea desde el Chat...,solicitud de asistencia en linea desde el chat...,solicitud de asistencia en linea desde el chat...
15393,Situación Reportada:\r\n\r\n-SE ATIENDE A CX E...,situacion reportada: -se atiende a cx en hd 20...,situacion reportada: -
24657,"Situación Reportada:\r\n\r\nse llama a cx, par...","situacion reportada: se llama a cx, para indic...","situacion reportada: se llama a cx, para indic..."
6629,HelpDesk generado desde una llamada telefónica,helpdesk generado desde una llamada telefonica,helpdesk generado desde una llamada telefonica
5125,SSE TOMA LLAMADO DE LINEA\r\nCX INDICA QUE NEC...,sse toma llamado de linea cx indica que necesi...,sse toma llamado de linea cx indica que necesi...


In [ ]:
# =========================
# 4.5 Eliminar textos analíticos vacíos
# =========================

df_work = df_work[df_work["len_texto_analitico"] > 0].reset_index(drop=True)

print("Registros finales para NLP:", df_work.shape[0])


Registros finales para NLP: 27477


In [ ]:
# =========================
# 4.6 Limpieza adicional de metadatos (chat/canal/fecha) - APLICACIÓN
# =========================

def limpiar_metadatos(texto):
    t = str(texto)

    # eliminar plantillas/metadatos típicos
    for p in patrones_metadatos:
        t = re.sub(p, " ", t)

    # eliminar fechas comunes (varios formatos)
    t = re.sub(patron_fechas, " ", t)

    # eliminar nombres/códigos que se detectaron como ruido
    t = re.sub(patron_nombres, " ", t)

    # limpieza final de espacios
    t = re.sub(r"\s+", " ", t).strip()
    return t

# crear texto_modelo desde texto_analitico
df_work["texto_modelo"] = df_work["texto_analitico"].apply(limpiar_metadatos)
df_work["len_texto_modelo"] = df_work["texto_modelo"].str.len()

df_work["len_texto_modelo"].describe()


,len_texto_modelo
count,27477.000000
mean,124.818030
std,166.094863
min,0.000000
25%,34.000000
50%,66.000000
75%,158.000000
max,2933.000000


In [ ]:
# =========================
# 4.6.1 Eliminar textos vacíos post-metadatos
# =========================

antes = df_work.shape[0]
df_work = df_work[df_work["len_texto_modelo"] > 0].reset_index(drop=True)
despues = df_work.shape[0]

print("Registros antes 4.6:", antes)
print("Registros después 4.6:", despues)

df_work.sample(5, random_state=99)[["Descripcion", "texto_analitico", "texto_modelo"]]


Registros antes 4.6: 27477
Registros después 4.6: 23852


,Descripcion,texto_analitico,texto_modelo
12933,Helpdesk duplicado de Gestión N° 2072447,helpdesk duplicado de gestion ndeg 2072447,helpdesk duplicado de gestion ndeg 2072447
22417,Situación Reportada:\r\n\r\n-se toma llamado \...,situacion reportada: -se toma llamado -cx soli...,situacion reportada: -se toma llamado -cx soli...
21871,Situación Reportada:\r\n\r\n-se atiende llamad...,situacion reportada: -,situacion reportada: -
9526,Situación Reportada:\r\nSE RECIBE LLAMADA\r\nI...,situacion reportada: se recibe llamada indicac...,situacion reportada: se recibe llamada indicac...
12691,se toma llamado de la linea\r\ncx idnica que t...,se toma llamado de la linea cx idnica que tien...,se toma llamado de la linea cx idnica que tien...


In [ ]:
# =========================
# 5.0 Muestra para modelado de tópicos
# =========================

N_MUESTRA = 8000

df_sample = df_work.sample(n=N_MUESTRA, random_state=42).reset_index(drop=True)
corpus_sample = df_sample["texto_modelo"].tolist()

print("Muestra:", len(corpus_sample))
print("Ejemplos:", corpus_sample[:2])


Muestra: 8000
Ejemplos: ['situacion reportada: -', 'se toma llamado de la liena cx indica que tiene problemas con centralizacion de sueldos necesita le revisen la centralizacion se coordin asesoria']


In [ ]:
# =========================
# 5.1 Modelo de embeddings
# =========================

from sentence_transformers import SentenceTransformer

modelo_embeddings = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
print("Modelo cargado")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo cargado


In [ ]:
# =========================
# 5.2 Embeddings
# =========================

embeddings_sample = modelo_embeddings.encode(
    corpus_sample,
    show_progress_bar=True,
    batch_size=128,
    normalize_embeddings=True
)

embeddings_sample.shape


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

(8000, 384)

In [ ]:
# =========================
# 6.1 Configuración BERTopic
# =========================

from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan

umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=30,
    min_samples=10,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),
    stop_words=stopwords.words("spanish"),
    min_df=10
)

topic_model = BERTopic(
    embedding_model=None,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    language="spanish",
    calculate_probabilities=True,
    verbose=True
)


In [ ]:
# =========================
# 6.2 Entrenamiento
# =========================

topics, probs = topic_model.fit_transform(corpus_sample, embeddings_sample)

topic_info = topic_model.get_topic_info()
topic_info.head(20)


2026-01-26 13:17:04,444 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 13:18:15,272 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:18:15,275 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:18:19,355 - BERTopic - Cluster - Completed ✓
2026-01-26 13:18:19,365 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:18:19,679 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1663,-1_cliente_cx_situacion_situacion reportada,"[cliente, cx, situacion, situacion reportada, ...",[situacion reportada: se toma llamado cx repor...
1,0,1261,0_asistencia_via_web_solicitud,"[asistencia, via, web, solicitud, soporte, lin...",[solicitud de asistencia en linea desde el cha...
2,1,778,1_cx_reporta_cx reporta_llamado cx,"[cx, reporta, cx reporta, llamado cx, reportad...",[situacion reportada: se toma llamado cx repor...
3,2,241,2_llama_llama cliente_contesta_volvera,"[llama, llama cliente, contesta, volvera, llam...","[se llama a cliente pero no contesta, se volve..."
4,3,226,3_hd_cierre_deriva_devuelve,"[hd, cierre, deriva, devuelve, vpp, ticket, si...","[situacion reportada: visto en hd 2077804, sit..."
5,4,172,4_cloud_area_empresa_cx,"[cloud, area, empresa, cx, clave, acceso, ambi...",[situacion reportada: -se toma llamado en turn...
6,5,157,5_cliente situacion_cliente_reportada indicaci...,"[cliente situacion, cliente, reportada indicac...",[situacion reportada: indicaciones: contactar ...
7,6,136,6_correo_envio_correos_configuracion,"[correo, envio, correos, configuracion, enviar...",[cliente no puede enviar correo a sus clientes...
8,7,112,7_periodo_contable_reproceso_abrir,"[periodo, contable, reproceso, abrir, realizar...",[situacion reportada: se toma llamado cx consu...
9,8,100,8_linea cx_llamado linea_cx indica_problemas,"[linea cx, llamado linea, cx indica, problemas...",[se toma llamado de la linea cx indica que tie...


In [ ]:
df_sample["topic"] = topics
df_sample["topic_name"] = df_sample["topic"].map(
    topic_info.set_index("Topic")["Name"]
)


In [ ]:
resumen_topicos = (
    df_sample
    .groupby(["topic", "topic_name"])
    .size()
    .reset_index(name="cantidad")
    .sort_values("cantidad", ascending=False)
)

resumen_topicos.head(15)


,topic,topic_name,cantidad
0,-1,-1_cliente_cx_situacion_situacion reportada,1663
1,0,0_asistencia_via_web_solicitud,1261
2,1,1_cx_reporta_cx reporta_llamado cx,778
3,2,2_llama_llama cliente_contesta_volvera,241
4,3,3_hd_cierre_deriva_devuelve,226
5,4,4_cloud_area_empresa_cx,172
6,5,5_cliente situacion_cliente_reportada indicaci...,157
7,6,6_correo_envio_correos_configuracion,136
8,7,7_periodo_contable_reproceso_abrir,112
9,8,8_linea cx_llamado linea_cx indica_problemas,100


In [ ]:
mapa_consultoria = {
    6: "Configuración técnica / Correos",
    7: "Gestión contable / Periodos",
    12: "Instalación y arquitectura",
    13: "Licencias y activaciones",
    14: "Desarrollo y requerimientos avanzados"
}


In [ ]:
df_sample["categoria_consultoria"] = df_sample["topic"].map(mapa_consultoria)
df_sample["categoria_consultoria"].value_counts(dropna=False)


,count
categoria_consultoria,
NaN,7491
Configuración técnica / Correos,136
Gestión contable / Periodos,112
Instalación y arquitectura,93
Licencias y activaciones,87
Desarrollo y requerimientos avanzados,81


8.1 Verificar columnas de módulos en la muestra

In [ ]:
[c for c in df_sample.columns if c.startswith("mod_")]


['mod_CW', 'mod_IW', 'mod_SW', 'mod_NW', 'mod_OW', 'mod_HW']

8.2 Construir una columna de módulo dominante

In [ ]:
mod_cols = [c for c in df_sample.columns if c.startswith("mod_")]

def modulo_dominante(row):
    mods = [c.replace("mod_", "") for c in mod_cols if row[c] == 1]
    if len(mods) == 1:
        return mods[0]
    elif len(mods) > 1:
        return "MULTI"
    else:
        return "NO_DEFINIDO"

df_sample["modulo_dominante"] = df_sample.apply(modulo_dominante, axis=1)

df_sample["modulo_dominante"].value_counts()


,count
modulo_dominante,
NO_DEFINIDO,3879
HW,2105
MULTI,848
NW,387
CW,280
SW,279
IW,184
OW,38


8.3 Cruce categoría de consultoría × módulo

In [ ]:
tabla_consultoria_modulo = (
    df_sample
    .dropna(subset=["categoria_consultoria"])
    .groupby(["categoria_consultoria", "modulo_dominante"])
    .size()
    .reset_index(name="cantidad")
    .sort_values("cantidad", ascending=False)
)

tabla_consultoria_modulo


,categoria_consultoria,modulo_dominante,cantidad
4,Configuración técnica / Correos,NO_DEFINIDO,92
11,Desarrollo y requerimientos avanzados,NO_DEFINIDO,70
31,Licencias y activaciones,NO_DEFINIDO,60
23,Instalación y arquitectura,NO_DEFINIDO,55
16,Gestión contable / Periodos,NO_DEFINIDO,47
13,Gestión contable / Periodos,CW,39
22,Instalación y arquitectura,MULTI,19
5,Configuración técnica / Correos,NW,12
15,Gestión contable / Periodos,MULTI,12
1,Configuración técnica / Correos,HW,11


8.4 Versión porcentual (para discurso ejecutivo)

In [ ]:
tabla_porcentual = (
    tabla_consultoria_modulo
    .assign(
        porcentaje=lambda x: x["cantidad"] / x.groupby("categoria_consultoria")["cantidad"].transform("sum") * 100
    )
    .sort_values(["categoria_consultoria", "porcentaje"], ascending=[True, False])
)

tabla_porcentual


,categoria_consultoria,modulo_dominante,cantidad,porcentaje
4,Configuración técnica / Correos,NO_DEFINIDO,92,67.647059
5,Configuración técnica / Correos,NW,12,8.823529
1,Configuración técnica / Correos,HW,11,8.088235
3,Configuración técnica / Correos,MULTI,9,6.617647
6,Configuración técnica / Correos,SW,7,5.147059
2,Configuración técnica / Correos,IW,4,2.941176
0,Configuración técnica / Correos,CW,1,0.735294
11,Desarrollo y requerimientos avanzados,NO_DEFINIDO,70,86.419753
10,Desarrollo y requerimientos avanzados,MULTI,4,4.938272
7,Desarrollo y requerimientos avanzados,CW,2,2.469136


Paso 9. Traducción directa a productos de consultoría y capacitación

In [ ]:
tabla_productos = tabla_porcentual.copy()

tabla_productos["producto_sugerido"] = tabla_productos["categoria_consultoria"].map({
    "Gestión contable / Periodos": "Curso y asesoría de cierres y reaperturas",
    "Configuración técnica / Correos": "Asesoría técnica de comunicaciones",
    "Instalación y arquitectura": "Servicio de implementación técnica",
    "Licencias y activaciones": "Asesoría de habilitación y licenciamiento",
    "Desarrollo y requerimientos avanzados": "Consultoría avanzada / servicio premium"
})

tabla_productos


,categoria_consultoria,modulo_dominante,cantidad,porcentaje,producto_sugerido
4,Configuración técnica / Correos,NO_DEFINIDO,92,67.647059,Asesoría técnica de comunicaciones
5,Configuración técnica / Correos,NW,12,8.823529,Asesoría técnica de comunicaciones
1,Configuración técnica / Correos,HW,11,8.088235,Asesoría técnica de comunicaciones
3,Configuración técnica / Correos,MULTI,9,6.617647,Asesoría técnica de comunicaciones
6,Configuración técnica / Correos,SW,7,5.147059,Asesoría técnica de comunicaciones
2,Configuración técnica / Correos,IW,4,2.941176,Asesoría técnica de comunicaciones
0,Configuración técnica / Correos,CW,1,0.735294,Asesoría técnica de comunicaciones
11,Desarrollo y requerimientos avanzados,NO_DEFINIDO,70,86.419753,Consultoría avanzada / servicio premium
10,Desarrollo y requerimientos avanzados,MULTI,4,4.938272,Consultoría avanzada / servicio premium
7,Desarrollo y requerimientos avanzados,CW,2,2.469136,Consultoría avanzada / servicio premium


----------------**CAPA 2:NIVEL FUNCIONAL POR MODULO
**-------------------------------

# =========================================================
# PASO 10 – CAPA 2: TOPICOS FUNCIONALES ESPECIFICOS PARA CW
# =========================================================


In [ ]:
# =========================================================
# PASO 10 – CAPA 2: TOPICOS FUNCIONALES ESPECIFICOS PARA CW
# =========================================================
# Diagnostico del error:
# KeyError 'modulo_dominante' significa que esa columna no existe en df_work.
# En tu pipeline, modulo_dominante se creo en df_sample (muestra), no en df_work.
# Solucion:
# 1) Verificar columnas disponibles.
# 2) Crear modulo_dominante en df_work usando los flags mod_*.
# 3) Filtrar CW (y opcionalmente filtrar por terminos contables si aun no existe categoria_consultoria en df_work).
# 4) Ejecutar BERTopic en el subconjunto CW.
# =========================================================

# =========================================================
# 10.0 Verificar columnas disponibles
# =========================================================
print("Columnas df_work (primeras 60):")
print(list(df_work.columns)[:60])

mod_cols = [c for c in df_work.columns if c.startswith("mod_")]
print("Columnas mod_* encontradas:", mod_cols)

if len(mod_cols) == 0:
    raise ValueError("No se encontraron columnas mod_*. Debes ejecutar el paso de deteccion de modulos (mod_CW, mod_IW, etc.) antes de este paso.")


# =========================================================
# 10.0.1 Crear modulo_dominante en df_work (si no existe)
# =========================================================
if "modulo_dominante" not in df_work.columns:
    def modulo_dominante_row(row):
        mods = [c.replace("mod_", "") for c in mod_cols if row[c] == 1]
        if len(mods) == 1:
            return mods[0]
        elif len(mods) > 1:
            return "MULTI"
        else:
            return "NO_DEFINIDO"

    df_work["modulo_dominante"] = df_work.apply(modulo_dominante_row, axis=1)

print("modulo_dominante creado. Distribucion:")
print(df_work["modulo_dominante"].value_counts().head(10))


# =========================================================
# 10.0.2 Asegurar que el texto a modelar exista (texto_modelo)
# =========================================================
if "texto_modelo" not in df_work.columns:
    raise ValueError("No existe la columna texto_modelo. Debes ejecutar el paso 4.6 (aplicacion) para crear texto_modelo antes de continuar.")

df_work["len_texto_modelo"] = df_work["texto_modelo"].astype(str).str.len()
df_work_base = df_work[df_work["len_texto_modelo"] > 0].reset_index(drop=True)


# =========================================================
# 10.1 Filtrar datos: CW (y contable si aplica)
# =========================================================
# Si ya tienes categoria_consultoria en df_work, filtramos por ella.
# Si no la tienes, hacemos un filtro contable por palabras clave (provisional) para no detener el pipeline.

tiene_categoria = "categoria_consultoria" in df_work_base.columns

if tiene_categoria:
    df_cw = df_work_base[
        (df_work_base["modulo_dominante"] == "CW") &
        (df_work_base["categoria_consultoria"] == "Gestión contable / Periodos")
    ].reset_index(drop=True)
    print("Filtro usado: modulo_dominante == CW y categoria_consultoria == Gestion contable / Periodos")
else:
    # Filtro provisional por keywords contables (ajustable)
    patron_contable = r"\b(periodo|periodos|cierre|reapertur|apertur|reproceso|comprobante|asiento|contab|libro|balance|conciliaci|banco|cartola)\b"
    df_cw = df_work_base[
        (df_work_base["modulo_dominante"] == "CW") &
        (df_work_base["texto_modelo"].str.contains(patron_contable, regex=True, na=False))
    ].reset_index(drop=True)
    print("Filtro usado: modulo_dominante == CW y keywords contables (provisional).")

print("Registros CW para sub-topicos:", df_cw.shape[0])
print(df_cw[["texto_modelo"]].head(3))


# =========================================================
# 10.2 Preparar corpus CW
# =========================================================
corpus_cw = df_cw["texto_modelo"].tolist()
print("Textos para modelado CW:", len(corpus_cw))

if len(corpus_cw) < 200:
    print("Aviso: muy pocos registros para topic modeling. Considera relajar el filtro contable o aumentar muestra.")


# =========================================================
# 10.3 Modelo de embeddings (si no existe ya en memoria)
# =========================================================
# Si ya cargaste modelo_embeddings en pasos anteriores, esto no lo reemplaza.
try:
    modelo_embeddings
    print("modelo_embeddings ya existe en memoria.")
except NameError:
    from sentence_transformers import SentenceTransformer
    modelo_embeddings = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    print("modelo_embeddings cargado.")


# =========================================================
# 10.4 Generar embeddings para CW
# =========================================================
embeddings_cw = modelo_embeddings.encode(
    corpus_cw,
    show_progress_bar=True,
    batch_size=128,
    normalize_embeddings=True
)
print("Shape embeddings CW:", embeddings_cw.shape)


# =========================================================
# 10.5 Configurar y entrenar BERTopic para sub-topicos CW
# =========================================================
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan
from nltk.corpus import stopwords

umap_cw = umap.UMAP(
    n_neighbors=10,
    n_components=5,
    min_dist=0.1,
    metric="cosine",
    random_state=42
)

hdbscan_cw = hdbscan.HDBSCAN(
    min_cluster_size=15,
    min_samples=5,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

vectorizer_cw = CountVectorizer(
    ngram_range=(1, 2),
    stop_words=stopwords.words("spanish"),
    min_df=5
)

topic_model_cw = BERTopic(
    embedding_model=None,
    umap_model=umap_cw,
    hdbscan_model=hdbscan_cw,
    vectorizer_model=vectorizer_cw,
    language="spanish",
    calculate_probabilities=True,
    verbose=True
)

topics_cw, probs_cw = topic_model_cw.fit_transform(corpus_cw, embeddings_cw)

topic_info_cw = topic_model_cw.get_topic_info()
print("Topicos encontrados (incluye -1):", topic_info_cw.shape[0])
topic_info_cw.head(20)


# =========================================================
# 10.6 Asignar sub-topico funcional CW al dataset y resumir
# =========================================================
df_cw["subtopico_cw"] = topics_cw
df_cw["subtopico_cw_nombre"] = df_cw["subtopico_cw"].map(
    topic_info_cw.set_index("Topic")["Name"]
)

resumen_cw = (
    df_cw
    .groupby(["subtopico_cw", "subtopico_cw_nombre"])
    .size()
    .reset_index(name="cantidad")
    .sort_values("cantidad", ascending=False)
)

print("Resumen sub-topicos CW (top 20):")
print(resumen_cw.head(20))

# Opcional: ver tamaño del ruido -1
ruido = resumen_cw[resumen_cw["subtopico_cw"] == -1]["cantidad"].sum()
print("Cantidad en sub-topico -1 (ruido):", int(ruido))


Columnas df_work (primeras 60):
['CodHw', 'Descripcion', 'len_texto', 'texto_norm', 'len_texto_norm', 'flag_ia', 'flag_qa', 'flag_cx', 'flag_gestion', 'flag_responsable', 'mod_CW', 'mod_IW', 'mod_SW', 'mod_NW', 'mod_OW', 'mod_HW', 'texto_analitico', 'len_texto_analitico', 'texto_modelo', 'len_texto_modelo']
Columnas mod_* encontradas: ['mod_CW', 'mod_IW', 'mod_SW', 'mod_NW', 'mod_OW', 'mod_HW']
modulo_dominante creado. Distribucion:
modulo_dominante
NO_DEFINIDO    11807
HW              6247
MULTI           2409
NW              1084
CW               840
SW               798
IW               518
OW               149
Name: count, dtype: int64
Filtro usado: modulo_dominante == CW y keywords contables (provisional).
Registros CW para sub-topicos: 301
                                        texto_modelo
0  se hace comparativo libro rem.v/s comprobante ...
1  en apoyo a jss, se indica como crear un asient...
2  situacion reportada: ingresamos comprobante ma...
Textos para modelado CW: 301
mod

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2026-01-26 13:40:28,793 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Shape embeddings CW: (301, 384)


2026-01-26 13:40:33,942 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:40:33,943 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:40:33,972 - BERTopic - Cluster - Completed ✓
2026-01-26 13:40:33,977 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:40:34,011 - BERTopic - Representation - Completed ✓


Topicos encontrados (incluye -1): 5
Resumen sub-topicos CW (top 20):
   subtopico_cw                   subtopico_cw_nombre  cantidad
1             0       0_cx_contable_situacion_proceso       118
2             1     1_cx_comprobante_indica_reportada        71
0            -1   -1_comprobante_cliente_cx_situacion        68
3             2   2_contable_entrega_cliente_realizar        24
4             3  3_comprobante_cliente_error_respaldo        20
Cantidad en sub-topico -1 (ruido): 68


In [ ]:
# =========================================================
# PASO 10.8 – CORRECCION DEL ERROR max_df < min_df Y MEJORA PARA OBTENER TOPICOS MAS ESPECIFICOS EN CW
# =========================================================
# Diagnostico:
# El error "max_df corresponds to < documents than min_df" ocurre cuando CountVectorizer
# queda con un vocabulario demasiado pequeno por el tamano del subconjunto o por stop_words.
# En la practica pasa cuando:
# - hay muchos tokens muy repetidos (cx, cliente, situacion reportada) que se filtran,
# - y el min_df queda alto para la variedad real del texto.
#
# Solucion:
# 1) Usar un vectorizer mas tolerante: min_df menor (2) y max_df fijo (0.9).
# 2) Excluir MULTI_FAMILIA del topic modeling (es mezcla; no ayuda a especificidad).
# 3) Enfocarnos en COMPROBANTES_ASIENTOS y PERIODOS_CIERRE (tienen volumen suficiente).
# 4) Crear stopwords custom del dominio helpdesk para evitar topicos genericos.
# 5) Para BANCOS_CONCILIACION hay muy pocos casos (9): se requiere ampliar el filtro CW
#    o hacer busqueda dirigida de casos bancarios dentro de CW.
# =========================================================

import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan

# =========================================================
# 10.8.3 Stopwords custom (dominio helpdesk) para mejorar especificidad
# =========================================================
stop_es = set(stopwords.words("spanish"))

stop_custom = {
    "cx","cliente","clientes","situacion","reportada","indica","indican","se","toma","llamado",
    "solicitud","caso","ticket","helpdesk","hd","softland","cl","sr","sra","estimado","estimada",
    "gracias","favor","adjunta","adjunto","correo","mail","contactar","contacto","respuesta",
    "informa","informe","informo","enviado","envia","envio","recibe","recibido","deriva","derivado",
    "cierre","cierra"  # ojo: si quieres topicos de cierre, comenta estas dos lineas
}

stop_total = list(stop_es.union(stop_custom))

# =========================================================
# 10.8.4 Funcion para ejecutar BERTopic en una familia especifica (robusta)
# =========================================================
def ejecutar_bertopic_familia(df_fam, nombre_familia):
    n = df_fam.shape[0]
    print("\nFamilia:", nombre_familia, "| Registros:", n)

    corpus_fam = df_fam["texto_modelo"].astype(str).tolist()

    embeddings_fam = modelo_embeddings.encode(
        corpus_fam,
        show_progress_bar=True,
        batch_size=64,
        normalize_embeddings=True
    )

    umap_fam = umap.UMAP(
        n_neighbors=8,
        n_components=5,
        min_dist=0.1,
        metric="cosine",
        random_state=42
    )

    hdbscan_fam = hdbscan.HDBSCAN(
        min_cluster_size=10,
        min_samples=3,
        metric="euclidean",
        cluster_selection_method="eom",
        prediction_data=True
    )

    # Vectorizer mas tolerante: min_df bajo y max_df fijo para evitar el error
    vectorizer_fam = CountVectorizer(
        ngram_range=(1, 2),
        stop_words=stop_total,
        min_df=2,
        max_df=0.9
    )

    tm_fam = BERTopic(
        embedding_model=None,
        umap_model=umap_fam,
        hdbscan_model=hdbscan_fam,
        vectorizer_model=vectorizer_fam,
        language="spanish",
        calculate_probabilities=False,
        verbose=True
    )

    topics_fam, _ = tm_fam.fit_transform(corpus_fam, embeddings_fam)
    info_fam = tm_fam.get_topic_info()

    df_out = df_fam.copy()
    df_out["subtopico_familia"] = topics_fam
    df_out["subtopico_familia_nombre"] = df_out["subtopico_familia"].map(
        info_fam.set_index("Topic")["Name"]
    )

    resumen_fam = (
        df_out
        .groupby(["subtopico_familia", "subtopico_familia_nombre"])
        .size()
        .reset_index(name="cantidad")
        .sort_values("cantidad", ascending=False)
    )

    print("\nTopicos (top 10):")
    print(resumen_fam.head(10))

    ruido = int(resumen_fam[resumen_fam["subtopico_familia"] == -1]["cantidad"].sum())
    print("Cantidad en -1 (ruido):", ruido)

    return {"df": df_out, "topic_info": info_fam, "resumen": resumen_fam, "modelo": tm_fam}


# =========================================================
# 10.8.5 Ejecutar solo familias con volumen suficiente y evitar MULTI_FAMILIA
# =========================================================
resultados_familias = {}

# COMPROBANTES_ASIENTOS
df_comp = df_cw[df_cw["familia_cw"] == "COMPROBANTES_ASIENTOS"].reset_index(drop=True)
if df_comp.shape[0] >= 40:
    resultados_familias["COMPROBANTES_ASIENTOS"] = ejecutar_bertopic_familia(df_comp, "COMPROBANTES_ASIENTOS")
else:
    print("COMPROBANTES_ASIENTOS insuficiente:", df_comp.shape[0])

# PERIODOS_CIERRE
df_per = df_cw[df_cw["familia_cw"] == "PERIODOS_CIERRE"].reset_index(drop=True)
if df_per.shape[0] >= 40:
    resultados_familias["PERIODOS_CIERRE"] = ejecutar_bertopic_familia(df_per, "PERIODOS_CIERRE")
else:
    print("PERIODOS_CIERRE insuficiente:", df_per.shape[0])

print("\nFamilias procesadas:", list(resultados_familias.keys()))

# =========================================================
# 10.8.6 Nota estrategica sobre BANCOS_CONCILIACION
# =========================================================
# BANCOS_CONCILIACION tiene 9 registros: demasiado poco para topicos estables.
# Siguiente paso recomendado:
# ampliar recuperacion de casos bancarios dentro de CW con un filtro dirigido en todo df_work.
# (se ejecuta en el siguiente paso si lo autorizas)



Familia: COMPROBANTES_ASIENTOS | Registros: 112


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-26 13:54:08,992 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 13:54:09,195 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:54:09,197 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:54:09,207 - BERTopic - Cluster - Completed ✓
2026-01-26 13:54:09,211 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:54:09,233 - BERTopic - Representation - Completed ✓



Topicos (top 10):
   subtopico_familia                          subtopico_familia_nombre  \
1                  0  0_llamada indicaciones_ayuda_revisar_correlativo   
0                 -1      -1_puede_comprende_apoya_formato comprobante   
2                  1             1_pide_cuenta_cuadratura_verificacion   
3                  2                       2_ano_apertura_realiza_2024   
4                  3                   3_archivo_imagen_documento_solo   

   cantidad  
1        35  
0        22  
2        22  
3        22  
4        11  
Cantidad en -1 (ruido): 22

Familia: PERIODOS_CIERRE | Registros: 99


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-26 13:54:19,005 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 13:54:19,177 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:54:19,178 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:54:19,187 - BERTopic - Cluster - Completed ✓
2026-01-26 13:54:19,191 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:54:19,208 - BERTopic - Representation - Completed ✓



Topicos (top 10):
   subtopico_familia                           subtopico_familia_nombre  \
1                  0                   0_2025_solicita_proceso_realizar   
2                  1  1_entrega_informacion_generacion_entrega infor...   
3                  2            2_cvj_corrige_contacta_contable sistema   
0                 -1             -1_sistema_contabilidad_logra abrir_ia   

   cantidad  
1        50  
2        24  
3        15  
0        10  
Cantidad en -1 (ruido): 10

Familias procesadas: ['COMPROBANTES_ASIENTOS', 'PERIODOS_CIERRE']


In [ ]:
# =========================================================
# PASO 10.9 – FIX DEL ERROR AttributeError (embedding_model) Y REFINO DE REPRESENTACION
# =========================================================
# Diagnostico:
# Aunque pusimos embedding_model=None, con representation_model=KeyBERTInspired()
# BERTopic puede intentar usar embedding_model internamente para representacion
# (depende de version). Si embedding_model queda como None, intenta llamar
# self.embedding_model.embed_documents y falla.
#
# Solucion robusta:
# 1) NO usar KeyBERTInspired en esta etapa (para no depender del embedding_model interno).
# 2) En su lugar, usar un pipeline 100% estable:
#    - Mantener BERTopic con embedding_model=None (porque ya pasamos embeddings).
#    - Mejorar nombres usando get_topic() + c-TF-IDF (ya incluido).
#    - Y aplicar un post-procesamiento para renombrar topicos quitando palabras administrativas.
#
# Resultado:
# Topicos mas especificos sin romper el pipeline.
# =========================================================

import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan

# =========================================================
# 10.9.1 Stopwords extendidas (administrativas + anos) - estable
# =========================================================
stop_es = set(stopwords.words("spanish"))

stop_admin = {
    "cx","cliente","clientes","situacion","reportada","indica","indican","se","toma","llamado","llamada",
    "solicitud","solicita","solicitan","caso","ticket","helpdesk","hd","softland","cl","sr","sra",
    "gracias","favor","adjunta","adjunto","contactar","contacto","respuesta","informa","informo","informacion",
    "enviado","envia","envio","recibe","recibido","deriva","derivado","revisar","revisa","ayuda",
    "proceso","realizar","realiza","entrega","entregar","corrige","contacta","apoya","comprende"
}

stop_num = {"2023","2024","2025","2026","01","02","03","04","05","06","07","08","09","10","11","12"}

stop_total_ref = list(stop_es.union(stop_admin).union(stop_num))

token_pat = r"(?u)\b[a-zA-ZáéíóúñÁÉÍÓÚÑ]{2,}\b"

# =========================================================
# 10.9.2 Funcion BERTopic refinado SIN representation_model (fix)
# =========================================================
def bertopic_refinado_estable(corpus, nombre_familia, min_cluster_size=10):
    print("\nRefinado (estable) familia:", nombre_familia, "| Registros:", len(corpus))

    embeddings = modelo_embeddings.encode(
        corpus,
        show_progress_bar=True,
        batch_size=64,
        normalize_embeddings=True
    )

    um = umap.UMAP(
        n_neighbors=8,
        n_components=5,
        min_dist=0.1,
        metric="cosine",
        random_state=42
    )

    hdb = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=3,
        metric="euclidean",
        cluster_selection_method="eom",
        prediction_data=True
    )

    vec = CountVectorizer(
        ngram_range=(1, 2),
        stop_words=stop_total_ref,
        min_df=2,
        max_df=0.9,
        token_pattern=token_pat
    )

    tm = BERTopic(
        embedding_model=None,   # seguimos pasando embeddings explicitamente
        umap_model=um,
        hdbscan_model=hdb,
        vectorizer_model=vec,
        language="spanish",
        calculate_probabilities=False,
        verbose=True
    )

    topics, _ = tm.fit_transform(corpus, embeddings)
    info = tm.get_topic_info()

    return tm, topics, info


# =========================================================
# 10.9.3 Funcion para construir nombres mas utiles (top keywords por topico)
# =========================================================
def nombre_topico_por_keywords(topic_model, topic_id, top_n=6):
    if topic_id == -1:
        return "-1_RUIDO"
    palabras = topic_model.get_topic(topic_id)
    if palabras is None:
        return f"{topic_id}_SIN_PALABRAS"
    kw = [w for w, _ in palabras[:top_n]]
    # limpieza final por seguridad
    kw = [re.sub(r"\d+", "", x).strip() for x in kw if x.strip() != ""]
    kw = [x for x in kw if x not in stop_admin and x not in stop_num]
    return f"{topic_id}_" + "_".join(kw[:top_n])


def aplicar_nombres_utiles(topic_model, info_df):
    info_df2 = info_df.copy()
    info_df2["Nombre_funcional"] = info_df2["Topic"].apply(lambda t: nombre_topico_por_keywords(topic_model, t, top_n=6))
    return info_df2


# =========================================================
# 10.9.4 Ejecutar refinado estable para COMPROBANTES_ASIENTOS
# =========================================================
df_comp = resultados_familias["COMPROBANTES_ASIENTOS"]["df"].reset_index(drop=True)
corpus_comp = df_comp["texto_modelo"].astype(str).tolist()

tm_comp_ref, topics_comp_ref, info_comp_ref = bertopic_refinado_estable(
    corpus_comp, "COMPROBANTES_ASIENTOS", min_cluster_size=10
)

info_comp_ref2 = aplicar_nombres_utiles(tm_comp_ref, info_comp_ref)
print("\nCOMPROBANTES_ASIENTOS - top 15 con nombre funcional:")
print(info_comp_ref2[["Topic","Count","Name","Nombre_funcional"]].head(15))


# =========================================================
# 10.9.5 Ejecutar refinado estable para PERIODOS_CIERRE
# =========================================================
df_per = resultados_familias["PERIODOS_CIERRE"]["df"].reset_index(drop=True)
corpus_per = df_per["texto_modelo"].astype(str).tolist()

tm_per_ref, topics_per_ref, info_per_ref = bertopic_refinado_estable(
    corpus_per, "PERIODOS_CIERRE", min_cluster_size=10
)

info_per_ref2 = aplicar_nombres_utiles(tm_per_ref, info_per_ref)
print("\nPERIODOS_CIERRE - top 15 con nombre funcional:")
print(info_per_ref2[["Topic","Count","Name","Nombre_funcional"]].head(15))


# =========================================================
# 10.9.6 Guardar resultados en dataframes (para siguientes pasos)
# =========================================================
df_comp["subtopico_comp_ref"] = topics_comp_ref
df_per["subtopico_per_ref"] = topics_per_ref

# Mapas de nombres funcionales
map_comp = info_comp_ref2.set_index("Topic")["Nombre_funcional"].to_dict()
map_per = info_per_ref2.set_index("Topic")["Nombre_funcional"].to_dict()

df_comp["subtopico_comp_nombre"] = df_comp["subtopico_comp_ref"].map(map_comp)
df_per["subtopico_per_nombre"] = df_per["subtopico_per_ref"].map(map_per)

print("\nEjemplos COMPROBANTES_ASIENTOS:")
print(df_comp[["texto_modelo","subtopico_comp_ref","subtopico_comp_nombre"]].head(5))

print("\nEjemplos PERIODOS_CIERRE:")
print(df_per[["texto_modelo","subtopico_per_ref","subtopico_per_nombre"]].head(5))



Refinado (estable) familia: COMPROBANTES_ASIENTOS | Registros: 112


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-26 13:59:13,667 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 13:59:13,879 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:59:13,881 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:59:13,892 - BERTopic - Cluster - Completed ✓
2026-01-26 13:59:13,896 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:59:13,915 - BERTopic - Representation - Completed ✓



COMPROBANTES_ASIENTOS - top 15 con nombre funcional:
   Topic  Count                                    Name  \
0     -1     22  -1_puede_formato comprobante_mdo_fecha   
1      0     35        0_correlativo_libro_salto_visita   
2      1     22   1_pide_cuenta_cuadratura_verificacion   
3      2     22       2_ano_apertura_habia_comprobantes   
4      3     11         3_archivo_imagen_documento_solo   

                                    Nombre_funcional  
0                                           -1_RUIDO  
1          0_correlativo_libro_salto_visita_saber_bd  
2  1_pide_cuenta_cuadratura_verificacion_verifica...  
3       2_ano_apertura_habia_comprobantes_elimina_bd  
4          3_archivo_imagen_documento_solo_queda_ver  

Refinado (estable) familia: PERIODOS_CIERRE | Registros: 99


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-26 13:59:23,680 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 13:59:23,859 - BERTopic - Dimensionality - Completed ✓
2026-01-26 13:59:23,860 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 13:59:23,870 - BERTopic - Cluster - Completed ✓
2026-01-26 13:59:23,874 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 13:59:23,889 - BERTopic - Representation - Completed ✓



PERIODOS_CIERRE - top 15 con nombre funcional:
   Topic  Count                                               Name  \
0     -1     10             -1_sistema_contabilidad_logra abrir_ia   
1      0     50                          0_ano_saber_reporta_datos   
2      1     24   1_generacion_generacion periodo_generar_necesita   
3      2     15  2_cvj_reprocesar aperturas_contable sistema_ap...   

                                    Nombre_funcional  
0                                           -1_RUIDO  
1               0_ano_saber_reporta_datos_si_sistema  
2  1_generacion_generacion periodo_generar_necesi...  
3  2_cvj_reprocesar aperturas_contable sistema_ap...  

Ejemplos COMPROBANTES_ASIENTOS:
                                        texto_modelo  subtopico_comp_ref  \
0  situacion reportada: ingresamos comprobante ma...                   0   
1  situacion reportada: recuperamos bd verificamo...                  -1   
2  situacion reportada: indicaciones: revisar y v...             

In [ ]:
# =========================================================
# PASO 10.10 – CW PERIODOS_CIERRE: REFINAMIENTO PARA TOPICOS FUNCIONALES (ABRIR/CERRAR/REPROCESO)
# =========================================================
# Objetivo:
# Reducir el topico generico (ano/saber/datos) agregando stopwords mas fuertes
# y aumentar granularidad (min_cluster_size menor) para separar:
# - abrir/reabrir periodo
# - cierre de periodo
# - reproceso/centralizacion
# =========================================================

import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan

stop_es = set(stopwords.words("spanish"))

stop_admin_extra_periodos = {
    # genericas que dominaron el topico 0
    "ano","año","saber","datos","dato","sistema","contabilidad","contable","logra","si",
    # verbos/plantillas comunes
    "necesita","requiere","consultar","consulta","validar","valida","revisar","revisa","indica","indican",
    "cliente","clientes","cx","situacion","reportada","solicita","solicitud","proceso","realizar","realiza",
    "informa","informo","informacion","respuesta","contactar","contacto","ayuda","apoya","comprende",
    # numeros frecuentes
    "2023","2024","2025","2026","01","02","03","04","05","06","07","08","09","10","11","12"
}

stop_total_periodos = list(stop_es.union(stop_admin).union(stop_num).union(stop_admin_extra_periodos))

token_pat = r"(?u)\b[a-zA-ZáéíóúñÁÉÍÓÚÑ]{2,}\b"

df_per = resultados_familias["PERIODOS_CIERRE"]["df"].reset_index(drop=True)
corpus_per = df_per["texto_modelo"].astype(str).tolist()

embeddings_per = modelo_embeddings.encode(
    corpus_per,
    show_progress_bar=True,
    batch_size=64,
    normalize_embeddings=True
)

um = umap.UMAP(
    n_neighbors=8,
    n_components=5,
    min_dist=0.1,
    metric="cosine",
    random_state=42
)

hdb = hdbscan.HDBSCAN(
    min_cluster_size=8,
    min_samples=3,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

vec = CountVectorizer(
    ngram_range=(1, 2),
    stop_words=stop_total_periodos,
    min_df=2,
    max_df=0.9,
    token_pattern=token_pat
)

tm_per_v2 = BERTopic(
    embedding_model=None,
    umap_model=um,
    hdbscan_model=hdb,
    vectorizer_model=vec,
    language="spanish",
    calculate_probabilities=False,
    verbose=True
)

topics_per_v2, _ = tm_per_v2.fit_transform(corpus_per, embeddings_per)
info_per_v2 = tm_per_v2.get_topic_info()

def nombre_topico_keywords(modelo, topic_id, top_n=6):
    if topic_id == -1:
        return "-1_RUIDO"
    palabras = modelo.get_topic(topic_id) or []
    kw = [w for w, _ in palabras[:top_n]]
    kw = [re.sub(r"\d+", "", x).strip() for x in kw if x.strip() != ""]
    return f"{topic_id}_" + "_".join(kw[:top_n])

info_per_v2["Nombre_funcional"] = info_per_v2["Topic"].apply(lambda t: nombre_topico_keywords(tm_per_v2, t, top_n=6))

print("\nPERIODOS_CIERRE v2 - top 20:")
print(info_per_v2[["Topic","Count","Name","Nombre_funcional"]].head(20))

df_per["subtopico_per_v2"] = topics_per_v2
map_per_v2 = info_per_v2.set_index("Topic")["Nombre_funcional"].to_dict()
df_per["subtopico_per_v2_nombre"] = df_per["subtopico_per_v2"].map(map_per_v2)

print("\nDistribucion subtopicos v2:")
print(df_per["subtopico_per_v2_nombre"].value_counts().head(15))

print("\nEjemplos:")
print(df_per[["texto_modelo","subtopico_per_v2","subtopico_per_v2_nombre"]].head(10))


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-26 14:37:21,043 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 14:37:21,226 - BERTopic - Dimensionality - Completed ✓
2026-01-26 14:37:21,227 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 14:37:21,239 - BERTopic - Cluster - Completed ✓
2026-01-26 14:37:21,246 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 14:37:21,265 - BERTopic - Representation - Completed ✓



PERIODOS_CIERRE v2 - top 20:
   Topic  Count                                               Name  \
0     -1     13                   -1_opcion_abrir periodo_ia_abrir   
1      0     39            0_reporta_aperturar_cw_apertura periodo   
2      1     24  1_generacion_generacion periodo_generar_genera...   
3      2     15               2_cvj_abrir_abrir periodo_reprocesar   
4      3      8           3_generacion_base_revision_requerimiento   

                                    Nombre_funcional  
0                                           -1_RUIDO  
1  0_reporta_aperturar_cw_apertura periodo_llama_...  
2  1_generacion_generacion periodo_generar_genera...  
3  2_cvj_abrir_abrir periodo_reprocesar_periodo c...  
4  3_generacion_base_revision_requerimiento_elimi...  

Distribucion subtopicos v2:
subtopico_per_v2_nombre
0_reporta_aperturar_cw_apertura periodo_llama_puede                            39
1_generacion_generacion periodo_generar_generar periodo_puede_periodo puede    24
2_

In [ ]:
# =========================================================
# PASO 10.11 – CW BANCOS/CONCILIACION: AMPLIAR CASOS Y EXTRAER TOPICOS ESPECIFICOS
# =========================================================
# Objetivo:
# Obtener volumen suficiente para identificar topicos como "conciliacion bancaria" en CW.
# Se amplian casos buscando keywords bancarias en TODO df_work (incluye NO_DEFINIDO y MULTI),
# luego se prioriza los que tengan mod_CW=1 o modulo_dominante en (CW, MULTI).
# =========================================================

import re
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import umap
import hdbscan
from nltk.corpus import stopwords

# 10.11.1 Patron bancario amplio (ajustable)
pat_bancos = r"\b(conciliaci|conciliar|banco|bancos|cartola|cuenta corr|cuenta corriente|cheque|deposit|transfer|abono|cargo|tesorer|pago|pagos|movimiento banc|cuadratura banc|previred|recaud)\b"

df_bancos_raw = df_work[
    df_work["texto_modelo"].astype(str).str.contains(pat_bancos, regex=True, na=False)
].copy()

print("Registros con señal bancaria (todos los modulos):", df_bancos_raw.shape[0])

# 10.11.2 Priorizar casos con CW presente
# Incluimos:
# - mod_CW == 1 (mencion explicita CW)
# - o modulo_dominante en ("CW","MULTI") (cuando CW esta mezclado)
df_bancos_cw = df_bancos_raw[
    (df_bancos_raw["mod_CW"] == 1) | (df_bancos_raw["modulo_dominante"].isin(["CW","MULTI"]))
].reset_index(drop=True)

print("Registros bancarios priorizados a CW:", df_bancos_cw.shape[0])
print(df_bancos_cw[["modulo_dominante","texto_modelo"]].head(3))

# 10.11.3 Preparar corpus y embeddings
corpus_bancos = df_bancos_cw["texto_modelo"].astype(str).tolist()
emb_bancos = modelo_embeddings.encode(
    corpus_bancos,
    show_progress_bar=True,
    batch_size=64,
    normalize_embeddings=True
)

# 10.11.4 Vectorizer con stopwords refinadas (usar las mismas stop_total_ref)
stop_es = set(stopwords.words("spanish"))
stop_bancos_extra = {"cliente","clientes","cx","situacion","reportada","solicita","proceso","realizar","llamada","llamado","ayuda","revisar","revisa"}
stop_total_bancos = list(stop_es.union(stop_admin).union(stop_num).union(stop_bancos_extra))

vec_bancos = CountVectorizer(
    ngram_range=(1, 2),
    stop_words=stop_total_bancos,
    min_df=2,
    max_df=0.9,
    token_pattern=r"(?u)\b[a-zA-ZáéíóúñÁÉÍÓÚÑ]{2,}\b"
)

um_bancos = umap.UMAP(
    n_neighbors=8,
    n_components=5,
    min_dist=0.1,
    metric="cosine",
    random_state=42
)

hdb_bancos = hdbscan.HDBSCAN(
    min_cluster_size=8,
    min_samples=3,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

tm_bancos = BERTopic(
    embedding_model=None,
    umap_model=um_bancos,
    hdbscan_model=hdb_bancos,
    vectorizer_model=vec_bancos,
    language="spanish",
    calculate_probabilities=False,
    verbose=True
)

topics_bancos, _ = tm_bancos.fit_transform(corpus_bancos, emb_bancos)
info_bancos = tm_bancos.get_topic_info()

def nombre_kw(modelo, topic_id, top_n=6):
    if topic_id == -1:
        return "-1_RUIDO"
    palabras = modelo.get_topic(topic_id) or []
    kw = [w for w, _ in palabras[:top_n]]
    kw = [re.sub(r"\d+", "", x).strip() for x in kw if x.strip() != ""]
    return f"{topic_id}_" + "_".join(kw[:top_n])

info_bancos["Nombre_funcional"] = info_bancos["Topic"].apply(lambda t: nombre_kw(tm_bancos, t, 6))

print("\nCW BANCOS/CONCILIACION - top 20:")
print(info_bancos[["Topic","Count","Name","Nombre_funcional"]].head(20))

df_bancos_cw["subtopico_bancos"] = topics_bancos
map_bancos = info_bancos.set_index("Topic")["Nombre_funcional"].to_dict()
df_bancos_cw["subtopico_bancos_nombre"] = df_bancos_cw["subtopico_bancos"].map(map_bancos)

print("\nDistribucion subtopicos bancos (top 15):")
print(df_bancos_cw["subtopico_bancos_nombre"].value_counts().head(15))


Registros con señal bancaria (todos los modulos): 713
Registros bancarios priorizados a CW: 159
  modulo_dominante                                       texto_modelo
0            MULTI  se hace coenxion remota estadistica de ventas ...
1               CW  en apoyo a jss, se indica como crear un asient...
2            MULTI  situacion reportada: conversamos con cristian ...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2026-01-26 14:42:07,334 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 14:42:07,620 - BERTopic - Dimensionality - Completed ✓
2026-01-26 14:42:07,622 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 14:42:07,636 - BERTopic - Cluster - Completed ✓
2026-01-26 14:42:07,642 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 14:42:07,672 - BERTopic - Representation - Completed ✓



CW BANCOS/CONCILIACION - top 20:
   Topic  Count                                 Name  \
0     -1     13  -1_numero_tema_documento_base datos   
1      0     48   0_reporta_previred_area_movimiento   
2      1     45    1_comprobante_factura_debe_cheque   
3      2     37               2_rut_fono_email_cargo   
4      3      8             3_banco_cartola_chile_ia   
5      4      8              4_ia_ok_correo_capturar   

                                    Nombre_funcional  
0                                           -1_RUIDO  
1  0_reporta_previred_area_movimiento_negocio_are...  
2   1_comprobante_factura_debe_cheque_puede_auxiliar  
3               2_rut_fono_email_cargo_spa_asistente  
4                    3_banco_cartola_chile_ia_bd_mas  
5              4_ia_ok_correo_capturar_error_captura  

Distribucion subtopicos bancos (top 15):
subtopico_bancos_nombre
0_reporta_previred_area_movimiento_negocio_area negocio    48
1_comprobante_factura_debe_cheque_puede_auxiliar           4